In [76]:
import json
import jsonlines
from argparse import ArgumentParser
from collections import defaultdict as ddict
import os
from pathlib import Path
from utility import deter_if_harm
import numpy as np


import plotly.graph_objects as go
import numpy as np


def plot_ASR(data_l,victim_model,width =700):
	# 创建 Plotly 图表
	fig = go.Figure()
	for data in data_l:
		# 添加第一组数据的折线
		fig.add_trace(go.Scatter(x=data["False_Positive"], y=data["ASR"], mode='lines', name=data["name"]))

	# 更新图表布局
	fig.update_layout(title=f'ASR V.S. False Positive for victim_model = {victim_model}',
					xaxis_title='False Positive',
					yaxis_title='ASR',
					legend_title='Groups',
					yaxis=dict(range=[0, 1]),
					width=width)

	# 显示图表
	fig.show()

def find_min_corresponding_b(a, b):
    """
    Function to find the minimum value in 'b' at the positions where 'a' has 1.
    If 'a' has no 1s, it returns -1.
    """
    min_val = float('inf')
    found = False

    for i in range(len(a)):
        if a[i] == 1:
            found = True
            if b[i] < min_val:
                min_val = b[i]

    return min_val if found else 0

def analysis_ASR(args):
	q_s_harm = ddict( lambda: ddict(list))
	q_s = []
	target_lm_generations = []
	p_s = []
	harm_scores = []
	ppl_scores = []
	with jsonlines.open(args.path) as f:
		for line in f:
			q_s.append(line["q"])
			harm_scores.append(line["reward"])
			try:
				ppl_value = line["ppl_score"]
			except:
				ppl_value = line["ppl_q_p"]
				
			target_lm_generations.append(line["target_lm_generation"])
			q_s_harm[line["q"]]["harm_scores"].append(line["reward"])
			q_s_harm[line["q"]]["target_lm_generations"].append(line["target_lm_generation"])
			if ppl_value is not None:
				q_s_harm[line["q"]]["ppl"].append(ppl_value)
				ppl_scores.append(ppl_value)

	all_harms = deter_if_harm(harm_scores,target_lm_generations,determine_way=args.determine_way)
	# print('len(all_harms)',len(all_harms))
	
	all_harms_over_qs = []
	q_ppl = []
	# print('len(q_s_harm)',len(q_s_harm))
	for q in q_s_harm:
		# print(len(q_s_harm[q]["harm_scores"]))
		q_harms = deter_if_harm(q_s_harm[q]["harm_scores"],q_s_harm[q]["target_lm_generations"],determine_way=args.determine_way)
		if any(q_harms):
			all_harms_over_qs.append(1)
		else:
			# print(q)
			all_harms_over_qs.append(0)
		if "ppl" in q_s_harm[q]:
			q_ppl.append(find_min_corresponding_b(q_harms,q_s_harm[q]["ppl"]))

	# print("asr_over_all_instances",sum(all_harms)/len(all_harms))
	# print("asr_over_all_qs",sum(all_harms_over_qs)/len(all_harms_over_qs))
	# # print(q_ppl)
	# print('sum(all_harms_over_qs)',sum(all_harms_over_qs))


	q_ppl_wo_zero = [_ for _ in q_ppl if _ != 0 ]

	with open(args.base_ppl) as f:
		base_ppl = json.load(f)
	base_ppl_values = list(sorted(list(base_ppl.values()),reverse= True))
	# print(len(base_ppl_values))
	False_Positive_l = []
	ASR_l = []
	for index,threshold_ppl in enumerate(base_ppl_values):
		# when > (instead of greater or equal) than threshold, then take it as the one filtered by ppl detecotr
		False_Positive = index / len(base_ppl_values)
		ASR = len([_ for _ in q_ppl_wo_zero if _ <= threshold_ppl]) / len(q_ppl)
		False_Positive_l.append(False_Positive)
		ASR_l.append(ASR)
		# print(False_Positive,"|",ASR)
		# if index == len(base_ppl_values)-1:
		# 	print(threshold_ppl,'threshold_ppl')
	return False_Positive_l,ASR_l

In [63]:
import numpy as np
def sample_l(a,b,sample = 50):

	# Assuming 'a' and 'b' are numpy arrays with 520 elements each
	# We can use numpy's linspace to generate 100 evenly spaced indices
	indices = np.linspace(0, len(a)-1, sample, dtype=int)

	# Now, we sample elements from 'a' and 'b' using these indices

	a_sampled = np.array(a)[indices]
	b_sampled = np.array(b)[indices]

	# Converting numpy arrays to lists before returning
	a_sampled_list = a_sampled.tolist()
	b_sampled_list = b_sampled.tolist()
	a_sampled_list.append(a[-1])
	b_sampled_list.append(b[-1])

	return a_sampled_list, b_sampled_list


In [49]:
class Args:
    def __init__(self,path,determine_way,base_ppl):
        self.path = path
        self.determine_way = determine_way
        self.base_ppl = base_ppl

In [77]:
victim_model = "llama2-7b-chat"
determine_way = "all"
base_ppl = f"/users/PAA0201/lzy37ld/why_attack_lzy/data/q_ppl/{victim_model}/q_ppl_d.json"
data_l = []

data_l.append(\
	{"path": "/fs/ess/PAA0201/lzy37ld/why_attack/ckpt/prompter_victim=llama2-7b-chat_prompt_type=q_p_model_name=llama2-base_sample_way_and_n_sample=loss_100_nsample=200_epoch_5/checkpoint-25000-val/val|prompter_val_checkpoint|decode_top_p_numreturn_50|promptway_prompter_q_p|q_prefix_long/llama2-7b-chat|max_new_tokens_100/targetlm.jsonl"\
  	,
  	"name":"top_p+50|q_prefix=long"}
	)

data_l.append(\
	{"path": "/fs/ess/PAA0201/lzy37ld/why_attack/ckpt/prompter_victim=llama2-7b-chat_prompt_type=q_p_model_name=llama2-base_sample_way_and_n_sample=loss_100_nsample=200_epoch_5/checkpoint-25000-val/val|prompter_val_checkpoint|decode_top_p_numreturn_50|promptway_prompter_q_p|q_prefix_medium/llama2-7b-chat|max_new_tokens_100/targetlm.jsonl"\
  	,
  	"name":"top_p+50|q_prefix=medium"}
	)

data_l.append(\
	{"path": "/fs/ess/PAA0201/lzy37ld/why_attack/ckpt/prompter_victim=llama2-7b-chat_prompt_type=q_p_model_name=llama2-base_sample_way_and_n_sample=loss_100_nsample=200_epoch_5/checkpoint-25000-val/val|prompter_val_checkpoint|decode_top_p_numreturn_50|promptway_prompter_q_p|q_prefix_short/llama2-7b-chat|max_new_tokens_100/targetlm.jsonl"\
  	,
  	"name":"top_p+50|q_prefix=short"}
	)

data_l.append(\
	{"path": "/fs/ess/PAA0201/lzy37ld/why_attack/ckpt/prompter_victim=llama2-7b-chat_prompt_type=q_p_model_name=llama2-base_sample_way_and_n_sample=loss_100_nsample=200_epoch_5/checkpoint-25000-val/val|prompter_val_checkpoint|decode_top_p_numreturn_50|promptway_prompter_q_p|q_rep_3/llama2-7b-chat|max_new_tokens_100/targetlm.jsonl"\
  	,
  	"name":"top_p+50|q_rep=3"}
	)
data_l.append(\
	{"path": "/fs/ess/PAA0201/lzy37ld/why_attack/ckpt/prompter_victim=llama2-7b-chat_prompt_type=q_p_model_name=llama2-base_sample_way_and_n_sample=loss_100_nsample=200_epoch_5/checkpoint-25000-val/val|prompter_val_checkpoint|decode_top_p_numreturn_50|promptway_prompter_q_p|q_rep_4/llama2-7b-chat|max_new_tokens_100/targetlm.jsonl"\
  	,
  	"name":"top_p+50|q_rep=4"}
	)
data_l.append(\
	{"path": "/fs/ess/PAA0201/lzy37ld/why_attack/ckpt/prompter_victim=llama2-7b-chat_prompt_type=q_p_model_name=llama2-base_sample_way_and_n_sample=loss_100_nsample=200_epoch_5/checkpoint-25000-val/val|prompter_val_checkpoint|decode_top_p_numreturn_50|promptway_prompter_q_p|q_rep_5/llama2-7b-chat|max_new_tokens_100/targetlm.jsonl"\
  	,
  	"name":"top_p+50|q_rep=5"}
	)
data_l.append(\
	{"path": "/fs/ess/PAA0201/lzy37ld/why_attack/ckpt/prompter_victim=llama2-7b-chat_prompt_type=q_p_model_name=llama2-base_sample_way_and_n_sample=loss_100_nsample=200_epoch_5/checkpoint-25000-val/val|prompter_val_checkpoint|decode_top_p_numreturn_50|promptway_prompter_q_p|q_rep_6/llama2-7b-chat|max_new_tokens_100/targetlm.jsonl"\
  	,
  	"name":"top_p+50|q_rep=6"}
	)

for data in data_l:
	assert f"prompter_victim={victim_model}" in data["path"]
	d = dict(path = data["path"],determine_way = determine_way, base_ppl = base_ppl)
	args = Args(**d)
	False_Positive_l,ASR_l = analysis_ASR(args)
	data.update({"False_Positive":False_Positive_l,"ASR":ASR_l})
	

plot_ASR(data_l,victim_model)



q_rep is better to both keeping low ppl while achieve high ASR

In [78]:
victim_model = "llama2-7b-chat"
determine_way = "all"
base_ppl = f"/users/PAA0201/lzy37ld/why_attack_lzy/data/q_ppl/{victim_model}/q_ppl_d.json"
data_l = []

data_l.append(\
	{"path": "/fs/ess/PAA0201/lzy37ld/why_attack/ckpt/prompter_victim=llama2-7b-chat_prompt_type=q_p_model_name=llama2-base_sample_way_and_n_sample=loss_100_nsample=200_epoch_5/checkpoint-25000-val/val|prompter_val_checkpoint|decode_top_p_numreturn_100|promptway_prompter_q_p|q_rep_6/llama2-7b-chat|max_new_tokens_100/targetlm.jsonl",
  	"name":"top_p+100|q_rep=6"}
	)

data_l.append(\
	{"path": "/fs/ess/PAA0201/lzy37ld/why_attack/ckpt/prompter_victim=llama2-7b-chat_prompt_type=q_p_model_name=llama2-base_sample_way_and_n_sample=loss_100_nsample=200_epoch_5/checkpoint-25000-val/val|prompter_val_checkpoint|decode_top_p_numreturn_100|promptway_prompter_q_p|q_rep_5/llama2-7b-chat|max_new_tokens_100/targetlm.jsonl",
  	"name":"top_p+100|q_rep=5"}
	)

data_l.append(\
	{"path": "/fs/ess/PAA0201/lzy37ld/why_attack/ckpt/prompter_victim=llama2-7b-chat_prompt_type=q_p_model_name=llama2-base_sample_way_and_n_sample=loss_100_nsample=200_epoch_5/checkpoint-25000-val/val|prompter_val_checkpoint|decode_top_p_numreturn_100|promptway_prompter_q_p|q_rep_4/llama2-7b-chat|max_new_tokens_100/targetlm.jsonl",
  	"name":"top_p+100|q_rep=4"}
	)

data_l.append(\
	{"path": "/fs/ess/PAA0201/lzy37ld/why_attack/ckpt/prompter_victim=llama2-7b-chat_prompt_type=q_p_model_name=llama2-base_sample_way_and_n_sample=loss_100_nsample=200_epoch_5/checkpoint-25000-val/val|prompter_val_checkpoint|decode_top_p_numreturn_100|promptway_prompter_q_p|q_rep_3/llama2-7b-chat|max_new_tokens_100/targetlm.jsonl",
  	"name":"top_p+100|q_rep=3"}
	)
还需要加入 top_p+200|q_rep = 4 ... etc..

for data in data_l:
	assert f"prompter_victim={victim_model}" in data["path"]
	d = dict(path = data["path"],determine_way = determine_way, base_ppl = base_ppl)
	args = Args(**d)
	False_Positive_l,ASR_l = analysis_ASR(args)
	data.update({"False_Positive":False_Positive_l,"ASR":ASR_l})

plot_ASR(data_l,victim_model)



Interesting cases,
Use prompt for unchanged harmful query still works for q_rep query.

In [81]:
victim_model = "llama2-7b-chat"
determine_way = "all"
base_ppl = f"/users/PAA0201/lzy37ld/why_attack_lzy/data/q_ppl/{victim_model}/q_ppl_d.json"
data_l = []

data_l.append(\
	{"path": "/fs/ess/PAA0201/lzy37ld/why_attack/ckpt/prompter_victim=llama2-7b-chat_prompt_type=q_p_model_name=llama2-base_sample_way_and_n_sample=loss_100_nsample=200_epoch_5/checkpoint-25000-val/val|prompter_val_checkpoint|decode_top_p_numreturn_100|promptway_prompter_q_p|q_rep_3|q_s_position_prompter_lm=raw|target_lm=processed/llama2-7b-chat|max_new_tokens_100/targetlm.jsonl"\
  	,
  	"name":"top_p+100|q_rep=3|prompter_lm_input=raw,target_lm_input=processed"}
	)
data_l.append(\
	{"path": "/fs/ess/PAA0201/lzy37ld/why_attack/ckpt/prompter_victim=llama2-7b-chat_prompt_type=q_p_model_name=llama2-base_sample_way_and_n_sample=loss_100_nsample=200_epoch_5/checkpoint-25000-val/val|prompter_val_checkpoint|decode_top_p_numreturn_100|promptway_prompter_q_p|q_rep_4|q_s_position_prompter_lm=raw|target_lm=processed/llama2-7b-chat|max_new_tokens_100/targetlm.jsonl"\
  	,
  	"name":"top_p+100|q_rep=4|prompter_lm_input=raw,target_lm_input=processed"}
	)
data_l.append(\
	{"path": "/fs/ess/PAA0201/lzy37ld/why_attack/ckpt/prompter_victim=llama2-7b-chat_prompt_type=q_p_model_name=llama2-base_sample_way_and_n_sample=loss_100_nsample=200_epoch_5/checkpoint-25000-val/val|prompter_val_checkpoint|decode_top_p_numreturn_100|promptway_prompter_q_p|q_rep_5|q_s_position_prompter_lm=raw|target_lm=processed/llama2-7b-chat|max_new_tokens_100/targetlm.jsonl"\
  	,
  	"name":"top_p+100|q_rep=5|prompter_lm_input=raw,target_lm_input=processed"}
	)


for data in data_l:
	assert f"prompter_victim={victim_model}" in data["path"]
	d = dict(path = data["path"],determine_way = determine_way, base_ppl = base_ppl)
	args = Args(**d)
	False_Positive_l,ASR_l = analysis_ASR(args)
	data.update({"False_Positive":False_Positive_l,"ASR":ASR_l})
	

plot_ASR(data_l,victim_model,width=1000)

